### Imports

In [ ]:
import cv2
import torch
from huggingface_hub import hf_hub_download
import numpy as np
import pandas as pd
from PIL import Image, ImageEnhance, ImageFilter
from PIL.ImageOps import invert
from own_utils import remove_overlapping_junctions, non_max_suppression_fast, create_window, apply_transformations

In [ ]:
show_intermediate = False   # set to True to show intermediate results

### Image preprocessing
Use the sliders to find the setting which best shows the drawn circuit, once you are satisfied press "Enter"

In [ ]:
test_image = '../assets/example_image.jpg'

In [ ]:
tf = create_window(test_image)
img = cv2.imread(test_image)
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = apply_transformations(gray_img, tf['contrast'], tf['blur'], tf['threshold'], tf['erode'], tf['dilate'], tf['invert'])

### Components Inference

In [ ]:
c_hf_model = hf_hub_download('Timdb/electronic-circuit-detection', 'components.pt')
c_model = torch.hub.load('ultralytics/yolov5', 'custom', c_hf_model)

In [ ]:
# Inference
c_results = c_model(img)

# Print and show results
if show_intermediate:
    print(c_results.pandas().xyxy)
    c_results.show()

### Junction inference

In [ ]:
j_hf_model = hf_hub_download('Timdb/electronic-circuit-detection', 'junctions.pt')
j_model = torch.hub.load('ultralytics/yolov5', 'custom', j_hf_model)

In [ ]:
# Perform inference on the image without components
j_results = j_model(img)

# Print and show results
if show_intermediate:
    print(j_results.pandas().xyxy)
    j_results.show()

### Post processing

In [ ]:
# Remove overlapping junctions
c_coords, j_coords = remove_overlapping_junctions(j_results, c_results, overlap_threshold=0.1)

# Remember the old junctions
columns = ['xmin', 'ymin', 'xmax', 'ymax', 'confidence', 'class']
old_j_df = pd.DataFrame(j_coords, columns=columns)
old_c_df = pd.DataFrame(c_coords, columns=columns)

# Perform non-maximum suppression on coords and junctions
c_coords = non_max_suppression_fast(c_coords, iou_threshold=0.5)
j_coords = non_max_suppression_fast(j_coords, iou_threshold=0.5)

# Create dataframes from the components and remaining junctions
c_df = pd.DataFrame(c_coords, columns=columns)
j_df = pd.DataFrame(j_coords, columns=columns)

if show_intermediate:
    print("Old components:")
    print(old_c_df)
    
    print("Components:")
    print(c_df)

    print("Old junctions:")
    print(old_j_df)

    print("NMS Junctions:")
    print(j_df)

### Show final detections

In [ ]:
# using c_df containing the components and j_df containing the junctions, draw all bounding boxes on the original image
img = cv2.imread(test_image)
c_labels = c_model.model.names
j_labels = j_model.model.names

# Draw components
for index, row in c_df.iterrows():
    xmin = int(row['xmin'])
    ymin = int(row['ymin'])
    xmax = int(row['xmax'])
    ymax = int(row['ymax'])
    cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

    label = c_labels[int(row['class'])]
    cv2.putText(img, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

# Draw junctions
for index, row in j_df.iterrows():
    xmin = int(row['xmin'])
    ymin = int(row['ymin'])
    xmax = int(row['xmax'])
    ymax = int(row['ymax'])
    cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)

    label = j_labels[int(row['class'])]
    cv2.putText(img, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

# Save the image
cv2.imwrite('../assets/output.jpg', img)

### Convert to generated image
Take the final output list and generate the digital circuit based on that

In [ ]:
# TODO